# Blockchain

Using JSON (javascript object notation) to trasnport data because lightweight, often used for server-> webpage data movement, 'self-describing'.

- Data in name value pairs
- Data seperated by commas
- Curly brackets hold objects
- Square brackets hold arrays

eg:
{
"employees":[

    {"firstName":"John", "lastName":"Doe"},
    {"firstName":"Anna", "lastName":"Smith"},
    {"firstName":"Peter", "lastName":"Jones"}
    
]
}

Obj employees is an array with 3 objects. Each obj is a record of a person witha first and last name. 

For the __HASH__, look at various crypto methods. For storing passwords and usernames, also look at salt (crypto). Each transaction in block is usually hashed, resulting in a hash or __merkel tree__.

__object.--dict--__ is a special attribute that returns the obj as a dictionary:



In [14]:
class MyClass(object):
    class_var = 1

    def __init__(self, i_vaw, b_var):
        self.i_var = i_vaw
        self.b_var = b_var

foo = MyClass(2, 4)
bar = MyClass(3, 6)

print (MyClass.__dict__)
print (foo.__dict__)
print (bar.__dict__)


{'__module__': '__main__', 'class_var': 1, '__init__': <function MyClass.__init__ at 0x7f7c0c2c1950>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'i_var': 2, 'b_var': 4}
{'i_var': 3, 'b_var': 6}


In [2]:
from hashlib import sha256
import json

class Block:
    def __init__(self, index, transac, timestap):
        """ Block constructor
            :param index: unique  block ID
            :param transac: list of transactions
            :param timestamp: Time generation of block"""
        self.index = index
        self.transac = transac
        self.timestamp = timestamp

    def hasher(block):
        """ Returns hash of block after convertnig
            to JSON string"""
        block_json_str = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_json_str.encode()).hexdigest()

In [7]:
data = b"Hello World"
sha256(data).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

In [8]:
data = b"Hello World"
sha256(data).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

In [9]:
data = b"Hello1 World"
sha256(data).hexdigest()

'b67ac8c5bbbe3e844cfda6eb8c93f224683d18f4b6ad9a8cb2877991165103e2'